In [1]:
import datetime

import PIL.Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import random
import requests
import csv

import numpy as np
import tkinter as tk
from tkinter import filedialog
import os
from PIL import Image, ImageTk


#まとめて関数化
def scrapy(self):
    #個人設定
    options = Options()
    options.binary_location = os.getcwd() + '/chromedriver'

    driver = webdriver.Chrome(os.getcwd() + '/chromedriver')
    driver.get('https://weibo.com/')



    # ID/PASSを入力
    #ログインID/PASSを入力
    login_ID = input("この情報は保存されません，不安な方はコードに直接入力してください\nまた，あらかじめWeiboでログインできることを確かめ"
                     "ておいてください\n \n ログイン用IDを入力してください:")
    login_PASS = input("ログイン用パスワードを入力してください:")

    username = driver.find_element_by_css_selector('#loginname')
    username.send_keys(str(login_ID)) #ここに直接IDを入力してもいいです
    password = driver.find_element_by_name('password')
    password.send_keys(str(login_PASS)) #ここに直接パスワードを入力してもいいです

    time.sleep(1)

    # ログインボタンをクリック
    login_btn = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a')
    login_btn.click()

    #QRコードをこの間に認証
    QR_auth = ''
    while QR_auth != "y": #yと入力されるまで確認をとる
        if QR_auth == "q":
            break
        # 文字を入力する
        QR_auth = input("QRコードを認証したらyと入力してください:\n終了する場合はqと入力してください")  # QR_authにyを読み込む


    #This code will take you to the search screen in the site and search for the keyword
    #driver.get('https://s.weibo.com/weibo?q=燕三条')



    #検索したいキーワードを入力
    query = input("検索したいキーワードを入力してください:\n終了する場合はcancelを押してください")

    #検索したいページに遷移
    driver.get('https://s.weibo.com/weibo?q='+str(query))

    tree = BeautifulSoup(driver.page_source, "html.parser")


    list_raw = []


    for element2 in tree.find_all(class_="txt"):
        list_raw.append(element2.text)
        #print(element2.text)


    '''
    for element1 in tree.find_all(class_="from"):
            print(element1.text)
    '''



    #ページ数取得
    count = tree.find(class_="s-scroll").find_all("li")


    """動作チェック用済み

    for element in count:
        print(element)


    """

    #ページ数
    pages = len(count) + 1 #rangeで１減らされるから足しておく

    #ページ数表示
    #print(len(count))







    for flip_pages in range(2,pages):
        driver.get('https://s.weibo.com/weibo?q='+ str(query) + '&page='+str(flip_pages))
        tree = BeautifulSoup(driver.page_source, "html.parser")
        for element2 in tree.find_all(class_="txt"):
            list_raw.append(element2.text)
            #print(element2.text)

        time.sleep(random.uniform(1,2))

    #This code retrieves the content of a tweet

    '''
    print(driver.page_source)
    '''


    #配列をクレンジング
    list_clear = [clear for clear in list_raw if '展开c' not in clear] #展开cを含む配列を削除
    list_clear = [clear.replace('\n','') for clear in list_clear]
    list_clear = [clear.replace('\u200b','') for clear in list_clear]
    list_clear = [clear.replace('\ue627','') for clear in list_clear]
    list_clear = [clear.replace('收起d','') for clear in list_clear]


    #配列に見出しを追加
    list_clear.insert(0,'ツイート内容')

    #print(list_clear)

    list_matrix = np.array(list_clear).reshape((1, len(list_clear)))
    #print(list_matrix)

    #print(list_matrix.transpose())



    '''
    #csvファイルの出力先を取得
    file_path = tkinter.filedialog.askdirectory(initialdir = os.getcwd())
    '''

    #csvファイル作成

    #日付を取得

    now = datetime.datetime.now()
    filename = 'weibodata_' + str(query) + '_'  + now.strftime('%Y_%m_%d_%H_%M_%S') + '.csv'

    with open(filename, 'w') as file:
        writer = csv.writer(file, lineterminator='\n')
        writer.writerow(list_matrix)

    driver.quit()

'''
#画像のリサイズ
def Image_resize(Imagename):
    img = Image.open(str(Imagename))
    img = img.resize(( 200, 200))
    img = ImageTk.PhotoImage(img)
    return img
'''



#tkinterよる画面設定

# Tkクラス生成
root = tk.Tk()
# 画面サイズ
root.geometry('500x500')
# 画面タイトル
root.title('スクレイピングくん')


# メインフレームの作成と設置
frame = tk.Frame(root)
frame.pack(fill = tk.BOTH, padx=20,pady=10)

#画像ファイルをインスタンス変数に代入
img = tk.PhotoImage(file="weibo_icon.png",master=root)
img1 = tk.PhotoImage(file="weibo_cat.png",master=root)

'''
img = PIL.Image.open('weibo_icon.png')
img = img.resize((200,200),PIL.Image.ANTIALIAS)
#画像を出力する
dispimg = PIL.ImageTk.PhotoImage(img)
'''





# 各種ウィジェットの作成
btn = tk.Button(frame, text="weiboでスクレイピング", image=img, compound="top")
btn1 = tk.Button(frame, text="今後追加予定", image=img1, compound="top")
btn2 = tk.Button(frame, text="今後追加予定", image=img1, compound="top")
btn3 = tk.Button(frame, text="今後追加予定", image=img1, compound="top")






# 各種ウィジェットの設置
btn.grid(row=0, column=0, padx=5)
btn1.grid(row=0, column=1, padx=5)
btn2.grid(row=1, column=0, padx=5)
btn3.grid(row=1, column=1, padx=5)

# ボタン表示
#btn.place(x=125, y=230, width=300, height=300)
# ボタンに関数をbind
btn.bind("<ButtonPress>", scrapy)

# tkinter表示
root.mainloop()




/var/folders/_0/k8cm7rxj5r1b69dfb804dgr80000gn/T/ipykernel_49447/1726683923.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + '/chromedriver')
/var/folders/_0/k8cm7rxj5r1b69dfb804dgr80000gn/T/ipykernel_49447/1726683923.py:36: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  username = driver.find_element_by_css_selector('#loginname')
/var/folders/_0/k8cm7rxj5r1b69dfb804dgr80000gn/T/ipykernel_49447/1726683923.py:38: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  password = driver.find_element_by_name('password')
/var/folders/_0/k8cm7rxj5r1b69dfb804dgr80000gn/T/ipykernel_49447/1726683923.py:44: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  login_btn = driver.find_element_by_xpat